In [1]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

In [2]:
data_loc = '../data/'

In [3]:
fighters = pd.read_csv(data_loc + 'update_list_fighters.csv')
fighters.tail(3)

,fighter,fighter_url
69,Victoria Leonardo,http://ufcstats.com/fighter-details/9dedfdd910...
70,Themba Gorimbo,http://ufcstats.com/fighter-details/40f3cb27fc...
71,Takashi Sato,http://ufcstats.com/fighter-details/8abac21874...


#### Extracting website data to data frame:

In [4]:
data = []
column_names = ['name','wins','losses','draws',
                'height','weight','reach','stance','dob',
                'slpm','str_acc','sapm','str_def','td_avg','td_acc','td_def','sub_avg',
                'last_update','last_match','url']

for url in tqdm(fighters.fighter_url):
    try:
        data.append(get_fighter_data(url=url))
    except:
        print('error on ' + url)
        continue

df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:38<00:00,  1.87it/s]


,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
69,Victoria Leonardo,9,6,0,"5'5""",125lbs.,"64""",Orthodox,"Feb16,1990",3.27,39%,4.70,46%,1.60,27%,71%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/9dedfdd910...
70,Themba Gorimbo,11,4,0,"6'1""",170lbs.,"77""",Orthodox,"Jan23,1991",1.80,59%,1.71,42%,3.47,50%,66%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/40f3cb27fc...
71,Takashi Sato,16,7,0,"5'10""",170lbs.,"73""",Southpaw,"Jun09,1990",2.21,41%,3.73,44%,0.76,37%,60%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/8abac21874...


#### Merge data with existing file: 

In [5]:
org = pd.read_csv(data_loc + 'ufc_fighters_stats.csv')
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2382,Aaron Phillips,12,5,0,"5'9""",135lbs.,"71""",Southpaw,"Aug05,1989",1.51,53%,3.66,43%,0.86,50%,44%,0.6,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/34c2b3656f...
2383,Joselyne Edwards,13,4,0,"5'8""",135lbs.,"70""",Orthodox,"Sep29,1995",5.30,52%,3.48,51%,0.33,15%,64%,0.2,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/f9ab66e672...
2384,Lucie Pudilova,14,8,0,"5'8""",135lbs.,"67""",Orthodox,"Jun13,1994",3.97,37%,3.95,51%,0.62,27%,63%,0.6,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/3cf66c62d9...


In [6]:
# drop rows that needs update from org
mask = ~org['name'].isin(df.name)
org = org[mask].reset_index(drop=True)
org.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2315,Aaron Phillips,12,5,0,"5'9""",135lbs.,"71""",Southpaw,"Aug05,1989",1.51,53%,3.66,43%,0.86,50%,44%,0.6,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/34c2b3656f...
2316,Joselyne Edwards,13,4,0,"5'8""",135lbs.,"70""",Orthodox,"Sep29,1995",5.30,52%,3.48,51%,0.33,15%,64%,0.2,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/f9ab66e672...
2317,Lucie Pudilova,14,8,0,"5'8""",135lbs.,"67""",Orthodox,"Jun13,1994",3.97,37%,3.95,51%,0.62,27%,63%,0.6,05-22-2023,04-15-2023,http://ufcstats.com/fighter-details/3cf66c62d9...


In [7]:
# combine original data frame with new one
data = pd.concat([org, df]).reset_index(drop=True) 
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2387,Victoria Leonardo,9,6,0,"5'5""",125lbs.,"64""",Orthodox,"Feb16,1990",3.27,39%,4.70,46%,1.60,27%,71%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/9dedfdd910...
2388,Themba Gorimbo,11,4,0,"6'1""",170lbs.,"77""",Orthodox,"Jan23,1991",1.80,59%,1.71,42%,3.47,50%,66%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/40f3cb27fc...
2389,Takashi Sato,16,7,0,"5'10""",170lbs.,"73""",Southpaw,"Jun09,1990",2.21,41%,3.73,44%,0.76,37%,60%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/8abac21874...


#### Saving file:

In [8]:
data.to_csv(data_loc + 'ufc_fighters_stats.csv', index=0)
data.tail(3)

,name,wins,losses,draws,height,weight,reach,stance,dob,slpm,str_acc,sapm,str_def,td_avg,td_acc,td_def,sub_avg,last_update,last_match,url
2387,Victoria Leonardo,9,6,0,"5'5""",125lbs.,"64""",Orthodox,"Feb16,1990",3.27,39%,4.70,46%,1.60,27%,71%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/9dedfdd910...
2388,Themba Gorimbo,11,4,0,"6'1""",170lbs.,"77""",Orthodox,"Jan23,1991",1.80,59%,1.71,42%,3.47,50%,66%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/40f3cb27fc...
2389,Takashi Sato,16,7,0,"5'10""",170lbs.,"73""",Southpaw,"Jun09,1990",2.21,41%,3.73,44%,0.76,37%,60%,0.0,06-11-2023,05-20-2023,http://ufcstats.com/fighter-details/8abac21874...
